In [47]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define the folder path
ir_assignment_folder = "/content/drive/MyDrive/IR Assignment"

Mounted at /content/drive


In [ ]:
# # CREATING DUMMY DATA FOR DEBUGGING

# import pandas as pd
# import numpy as np

# # Constants
# NUM_USER_FEATURES = 512
# NUM_BOOK_FEATURES = 512
# BOOKS_PER_USER = 5
# NUM_TRAIN_USERS = 1999
# NUM_TEST_USERS = 552
# TOTAL_USERS = NUM_TRAIN_USERS + NUM_TEST_USERS

# # Generate dummy user IDs
# user_ids = [f"User_{i+1}" for i in range(TOTAL_USERS)]

# # Function to generate random user features (same for each user)
# def generate_user_features():
#     return np.random.rand(NUM_USER_FEATURES)

# # Function to generate random book features (different for each book)
# def generate_book_features():
#     return np.random.rand(NUM_BOOK_FEATURES)

# # Generate dummy data
# data = []
# for user_id in user_ids:
#     user_features = generate_user_features()  # User features (same for all books)

#     for book_index in range(BOOKS_PER_USER):
#         book_isbn = f"ISBN_{np.random.randint(100000, 999999)}"  # Random ISBN
#         book_features = generate_book_features()  # Random book features
#         relevance = 1 if book_index < 2 else 0  # First 2 books = relevant, last 3 = not relevant

#         row = [user_id] + list(user_features) + [book_isbn] + list(book_features) + [relevance]
#         data.append(row)

# # Column names
# user_feature_cols = [f"UserFeature_{i}" for i in range(NUM_USER_FEATURES)]
# book_feature_cols = [f"BookFeature_{i}" for i in range(NUM_BOOK_FEATURES)]
# columns = ["User-ID"] + user_feature_cols + ["ISBN"] + book_feature_cols + ["Relevance"]

# # Create DataFrame
# df = pd.DataFrame(data, columns=columns)

# # Split into train and test
# df_train = df[df["User-ID"].isin(user_ids[:NUM_TRAIN_USERS])]
# df_test = df[df["User-ID"].isin(user_ids[NUM_TRAIN_USERS:])]

# # print(df_train)
# # print(df_test)

# # Save to CSV
# df_train.to_csv(f"{ir_assignment_folder}/new-user-embeddings/dummy-visual-train.csv", index=False)
# df_test.to_csv(f"{ir_assignment_folder}/new-user-embeddings/dummy-visual-test.csv", index=False)

# Baseline Model

In [48]:
import pandas as pd

baseline_train = pd.read_csv(f"{ir_assignment_folder}/new-user-embeddings/LTR-DATA/training_final_basic_embd.csv")
baseline_test = pd.read_csv(f"{ir_assignment_folder}/new-user-embeddings/LTR-DATA/testing_final_basic_embd.csv")

In [49]:
print(baseline_train.head(1))

     User_0    User_1    User_2    User_3    User_4    User_5    User_6  \
0  0.083739  0.003036  0.008076 -0.006699  0.083712  0.001526 -0.046294   

     User_7    User_8    User_9  ...  Book_248  Book_249  Book_250  Book_251  \
0 -0.083772 -0.039261 -0.007051  ... -0.083497 -0.035449 -0.083488 -0.083463   

   Book_252  Book_253  Book_254  Book_255        ISBN  Relevance  
0  0.083459  0.044558  -0.06231 -0.083461  0440224675          0  

[1 rows x 515 columns]


In [50]:
baseline_train = baseline_train.sort_values(by='User-ID')
baseline_test = baseline_test.sort_values(by='User-ID')

In [51]:
exclude_cols = ['User-ID','ISBN','Relevance']

feature_cols = [col for col in baseline_train.columns if col not in exclude_cols]

X_train_baseline = baseline_train[feature_cols]
y_train_baseline = baseline_train['Relevance']

X_test_baseline = baseline_test[feature_cols]
y_test_baseline = baseline_test['Relevance']

print(X_test_baseline.head(5))

     User_0    User_1    User_2    User_3    User_4    User_5    User_6  \
3  0.083681  0.001821  0.009279 -0.008342  0.083653  0.002751 -0.047821   
2  0.083681  0.001821  0.009279 -0.008342  0.083653  0.002751 -0.047821   
1  0.083681  0.001821  0.009279 -0.008342  0.083653  0.002751 -0.047821   
4  0.083681  0.001821  0.009279 -0.008342  0.083653  0.002751 -0.047821   
0  0.083681  0.001821  0.009279 -0.008342  0.083653  0.002751 -0.047821   

     User_7    User_8    User_9  ...  Book_246  Book_247  Book_248  Book_249  \
3 -0.083715 -0.041063 -0.005692  ... -0.083450  0.007466 -0.083488 -0.035600   
2 -0.083715 -0.041063 -0.005692  ... -0.083457  0.007345 -0.083495 -0.035512   
1 -0.083715 -0.041063 -0.005692  ... -0.083457  0.007261 -0.083495 -0.035459   
4 -0.083715 -0.041063 -0.005692  ... -0.083459  0.007313 -0.083497 -0.035488   
0 -0.083715 -0.041063 -0.005692  ... -0.083465  0.007206 -0.083503 -0.035411   

   Book_250  Book_251  Book_252  Book_253  Book_254  Book_255  
3 -0

In [52]:
group_train_baseline = baseline_train.groupby('User-ID').size().tolist()
group_test_baseline = baseline_test.groupby('User-ID').size().tolist()

In [53]:
from lightgbm import LGBMRanker

gbm_baseline = LGBMRanker(objective='lambdarank')
gbm_baseline.fit(X_train_baseline, y_train_baseline, group=group_train_baseline)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.182113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 130558
[LightGBM] [Info] Number of data points in the train set: 9935, number of used features: 512


LGBMRanker(objective='lambdarank')

In [54]:
# Create a df with the actual and predicted results
y_pred = gbm_baseline.predict(X_test_baseline)
y_actual = y_test_baseline.copy()

baseline_test_results = baseline_test[['User-ID', 'ISBN']].copy()
baseline_test_results['Actual'] = y_actual
baseline_test_results['Predicted'] = y_pred

print(baseline_test_results.head())

   User-ID        ISBN  Actual  Predicted
3     1131  0345370805       1   0.289432
2     1131  0060934417       1   0.701033
1     1131  0446610038       0   0.901178
4     1131  089480829X       0   0.070364
0     1131  0374216495       0   0.736682


In [55]:
# Compute mean NDCG@5
import numpy as np
from sklearn.metrics import ndcg_score

y_true_grouped = baseline_test_results.groupby("User-ID")["Actual"].apply(list)
y_pred_grouped = baseline_test_results.groupby("User-ID")["Predicted"].apply(list)

ndcg_values_baseline = [ndcg_score([true], [pred], k=5) for true, pred in zip(y_true_grouped, y_pred_grouped)]

mean_ndcg_5_baseline = np.mean(ndcg_values_baseline)

print(f"Mean NDCG@5: {mean_ndcg_5_baseline:.4f}")

Mean NDCG@5: 0.7588


In [56]:
# Compute mean Precision@2
def precision_at_k(y_true, y_pred, k=2):
    """Compute Precision@k"""
    sorted_indices = np.argsort(y_pred)[::-1][:k]  # Top-k items by prediction
    return np.mean(np.array(y_true)[sorted_indices])  # Precision = Relevant in Top-k / k

precision_values_baseline = [precision_at_k(true, pred, k=2) for true, pred in zip(y_true_grouped, y_pred_grouped)]

mean_precision_2_baseline = np.mean(precision_values_baseline)

print(f"Mean Precision@2: {mean_precision_2_baseline:.4f}")

Mean Precision@2: 0.4601


In [57]:
# Compute MAP@2
def average_precision_at_k(y_true, y_pred, k=2):
    """Compute Average Precision@k"""
    sorted_indices = np.argsort(y_pred)[::-1]
    relevant = np.array(y_true)[sorted_indices]

    precisions = [np.mean(relevant[:i+1]) for i in range(k) if relevant[i] == 1]
    return np.mean(precisions) if precisions else 0

ap_values_baseline = [average_precision_at_k(true, pred, k=2) for true, pred in zip(y_true_grouped, y_pred_grouped)]

mean_ap_2_baseline = np.mean(ap_values_baseline)

print(f"Mean Average Precision@2 (MAP@2): {mean_ap_2_baseline:.4f}")

Mean Average Precision@2 (MAP@2): 0.6116


# Visual Model


In [58]:
visual_train = pd.read_csv(f"{ir_assignment_folder}/new-user-embeddings/LTR-DATA/training_final_visual_embd.csv")
visual_test = pd.read_csv(f"{ir_assignment_folder}/new-user-embeddings/LTR-DATA/testing_final_visual_embd.csv")

In [59]:
visual_train = visual_train.sort_values(by='User-ID')
visual_test = visual_test.sort_values(by='User-ID')

In [61]:
exclude_cols = ['User-ID','ISBN','Relevance']

feature_cols = [col for col in visual_train.columns if col not in exclude_cols]

X_train_visual = visual_train[feature_cols]
y_train_visual = visual_train['Relevance']

X_test_visual = visual_test[feature_cols]
y_test_visual = visual_test['Relevance']

print(X_test_visual.head(1))

     User_0    User_1    User_2    User_3    User_4    User_5    User_6  \
3 -0.058409 -0.058465  0.006694 -0.058472  0.058469 -0.058489 -0.002654   

     User_7    User_8    User_9  ...  Book_502  Book_503  Book_504  Book_505  \
3 -0.058544 -0.058471  0.058428  ... -0.013173  0.029644 -0.003977   0.03445   

   Book_506  Book_507  Book_508  Book_509  Book_510  Book_511  
3  -0.00939 -0.058255 -0.058249  0.058229 -0.058234  -0.05822  

[1 rows x 1024 columns]


In [62]:
group_train_visual = visual_train.groupby('User-ID').size().tolist()
group_test_visual = visual_test.groupby('User-ID').size().tolist()

In [63]:
from lightgbm import LGBMRanker

gbm_visual = LGBMRanker(objective='lambdarank')
gbm_visual.fit(X_train_visual, y_train_visual, group=group_train_visual)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.235459 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 9935, number of used features: 1024


LGBMRanker(objective='lambdarank')

In [64]:
# Create a df with the actual and predicted results
y_pred = gbm_visual.predict(X_test_visual)
y_actual = y_test_visual.copy()

visual_test_results = visual_test[['User-ID', 'ISBN']].copy()
visual_test_results['Actual'] = y_actual
visual_test_results['Predicted'] = y_pred

print(visual_test_results.head())

   User-ID        ISBN  Actual  Predicted
3     1131  0345370805       1  -0.097077
2     1131  0060934417       1  -0.224405
1     1131  3453210719       0  -0.479145
4     1131  045120753X       0   1.085282
0     1131  3518368540       0   0.813535


In [65]:
# Compute mean NDCG@5
y_true_grouped = visual_test_results.groupby("User-ID")["Actual"].apply(list)
y_pred_grouped = visual_test_results.groupby("User-ID")["Predicted"].apply(list)

ndcg_values_visual = [ndcg_score([true], [pred], k=5) for true, pred in zip(y_true_grouped, y_pred_grouped)]

mean_ndcg_5_visual = np.mean(ndcg_values_visual)

print(f"Mean NDCG@5: {mean_ndcg_5_visual:.4f}")

Mean NDCG@5: 0.7848


In [66]:
# Compute mean Precision@2
precision_values_visual = [precision_at_k(true, pred, k=2) for true, pred in zip(y_true_grouped, y_pred_grouped)]

mean_precision_2_visual = np.mean(precision_values_visual)

print(f"Mean Precision@2: {mean_precision_2_visual:.4f}")

Mean Precision@2: 0.4973


In [67]:
# Compute MAP@2
ap_values_visual = [average_precision_at_k(true, pred, k=2) for true, pred in zip(y_true_grouped, y_pred_grouped)]

mean_ap_2_visual = np.mean(ap_values_visual)

print(f"Mean Average Precision@2 (MAP@2): {mean_ap_2_visual:.4f}")

Mean Average Precision@2 (MAP@2): 0.6797


# Statistical Tests

In [68]:
# T-test on NDCG
from scipy.stats import ttest_rel

min_len = min(len(ndcg_values_baseline), len(ndcg_values_visual))
baseline_ndcg = ndcg_values_baseline[:min_len]
visual_ndcg = ndcg_values_visual[:min_len]

t_stat, p_val = ttest_rel(baseline_ndcg, visual_ndcg)

print(f"Paired t-test results:")
print(f"  t-statistic = {t_stat:.4f}")
print(f"  p-value     = {p_val:.4f}")

if p_val < 0.05:
    print("✅ The difference is statistically significant (p < 0.05)")
else:
    print("⚠️ No statistically significant difference (p ≥ 0.05)")


Paired t-test results:
  t-statistic = -2.8428
  p-value     = 0.0046
✅ The difference is statistically significant (p < 0.05)


In [69]:
# T-test on Precision@2
from scipy.stats import ttest_rel

min_len = min(len(precision_values_baseline), len(precision_values_visual))
baseline_precision = precision_values_baseline[:min_len]
visual_precision = precision_values_visual[:min_len]

t_stat, p_val = ttest_rel(baseline_precision, visual_precision)

print(f"Paired t-test results:")
print(f"  t-statistic = {t_stat:.4f}")
print(f"  p-value     = {p_val:.4f}")

if p_val < 0.05:
    print("✅ The difference is statistically significant (p < 0.05)")
else:
    print("⚠️ No statistically significant difference (p ≥ 0.05)")

Paired t-test results:
  t-statistic = -2.1712
  p-value     = 0.0303
✅ The difference is statistically significant (p < 0.05)


In [70]:
# T-test on MAP@2
from scipy.stats import ttest_rel

min_len = min(len(ap_values_baseline), len(ap_values_visual))
baseline_aps = ap_values_baseline[:min_len]
visual_aps = ap_values_visual[:min_len]

t_stat, p_val = ttest_rel(baseline_aps, visual_aps)

print(f"Paired t-test results:")
print(f"  t-statistic = {t_stat:.4f}")
print(f"  p-value     = {p_val:.4f}")

if p_val < 0.05:
    print("✅ The difference is statistically significant (p < 0.05)")
else:
    print("⚠️ No statistically significant difference (p ≥ 0.05)")


Paired t-test results:
  t-statistic = -3.1039
  p-value     = 0.0020
✅ The difference is statistically significant (p < 0.05)
